In [1]:
import numpy as np
import skimage.io
import skimage
import matplotlib.pyplot as plt
import cv2
import numba
from flow2color import *
from tqdm import tqdm

In [2]:
# I(x,y)
I1, I2 = cv2.imread('yos08.pgm', 0), cv2.imread('yos09.pgm', 0)

# Apply Gaussian Blur
#size = 7
#I1, I2 = cv2.GaussianBlur(I1,(size, size),0), cv2.GaussianBlur(I2,(size, size),0)

# Stack the images -- I(x,y,t)
I = np.stack([I1, I2], axis=-1) / 255.0

gt = np.genfromtxt('yos_co.txt')

# Derivatives
Iy, Ix, It = np.gradient(I)

In [3]:
def flow2image(flow):
    res = np.zeros((flow.shape[0], flow.shape[1], 3), np.uint8)
    for i in range(flow.shape[0]):
        for j in range(flow.shape[1]):
            res[i,j] = np.uint8(flow2color(flow[i,j,0], flow[i,j,1]))
    return res

In [4]:
def epe(gt, y):
    return np.sqrt((y[:, 0]-gt[:,0])**2 + (y[:, 1]-gt[:,1])**2).mean()
def aae(gt, y):
    return np.rad2deg(np.arccos((y[:, 0] * gt[:, 0] + y[:, 1] * gt[:, 1] + 1) / np.sqrt((y[:, 0]**2 + y[:, 1]**2 + 1) * (gt[:, 0]**2 + gt[:, 1]**2 + 1))).mean())

In [5]:
@numba.jit(nopython=True)
def sor(Ix, Iy, It, psi1, psi_im1, psi_ip1, psi_jm1, psi_jp1, flow, alpha):
    
    w = 1.99
    ialpha = 1.0 / alpha
    
    # Iterate over u 
    for i in range(flow.shape[0]):
        for j in range(flow.shape[1]):
            b = - psi1[i,j] * ialpha * It[i,j,0] * Ix[i,j,0]
            # with a minus in the equation
            D = psi1[i,j] * ialpha * Ix[i,j,0] * Ix[i,j,0] + psi_im1[i,j] + psi_ip1[i,j] + psi_jm1[i,j] + psi_jp1[i,j]
            Lx = - psi_im1[i,j] * (flow[i-1,j,0] if i>0 else 0.0) - psi_jm1[i,j] * (flow[i,j-1,0] if j>0 else 0.0)
            Ux = psi1[i,j] * ialpha * Iy[i,j,0] * Ix[i,j,0] * flow[i,j,1] - psi_ip1[i,j] * (flow[i+1,j,0] if i<flow.shape[0]-1 else 0.0) - psi_jp1[i,j] * (flow[i,j+1,0] if j<flow.shape[1]-1 else 0.0)
            
            # with a plus in the equation
            #D = psi1[i,j] * ialpha * Ix[i,j,0] * Ix[i,j,0] - psi_im1[i,j] - psi_ip1[i,j] - psi_jm1[i,j] - psi_jp1[i,j]
            #Lx = psi_im1[i,j] * (flow[i-1,j,0] if i>0 else 0.0) + psi_jm1[i,j] * (flow[i,j-1,0] if j>0 else 0.0)
            #Ux = psi1[i,j] * ialpha * Iy[i,j,0] * Ix[i,j,0] * flow[i,j,1] + psi_ip1[i,j] * (flow[i+1,j,0] if i<flow.shape[0]-1 else 0.0) + psi_jp1[i,j] * (flow[i,j+1,0] if j<flow.shape[1]-1 else 0.0)
            
            flow[i,j,0] = (1-w) * flow[i,j,0] + w * ((b - Lx - Ux) / D)
            
    # Iterate over v
    for i in range(flow.shape[0]):
        for j in range(flow.shape[1]):
            b = - psi1[i,j] * ialpha * It[i,j,0] * Iy[i,j,0]
            # with a minus in the equation
            D = psi1[i,j] * ialpha * Iy[i,j,0] * Iy[i,j,0] + psi_im1[i,j] + psi_ip1[i,j] + psi_jm1[i,j] + psi_jp1[i,j]
            Lx = - psi_im1[i,j] * (flow[i-1,j,1] if i>0 else 0.0) - psi_jm1[i,j] * (flow[i,j-1,1] if j>0 else 0.0)
            Ux = psi1[i,j] * ialpha * Ix[i,j,0] * Iy[i,j,0] * flow[i,j,0] - psi_ip1[i,j] * (flow[i+1,j,1] if i<flow.shape[0]-1 else 0.0) - psi_jp1[i,j] * (flow[i,j+1,1] if j<flow.shape[1]-1 else 0.0)
            
            # with a plus in the equation
            #D = psi1[i,j] * ialpha * Iy[i,j,0] * Iy[i,j,0] - psi_im1[i,j] - psi_ip1[i,j] - psi_jm1[i,j] - psi_jp1[i,j]
            #Lx = psi_im1[i,j] * (flow[i-1,j,1] if i>0 else 0.0) + psi_jm1[i,j] * (flow[i,j-1,1] if j>0 else 0.0)
            #Ux = psi1[i,j] * ialpha * Ix[i,j,0] * Iy[i,j,0] * flow[i,j,0] + psi_ip1[i,j] * (flow[i+1,j,1] if i<flow.shape[0]-1 else 0.0) + psi_jp1[i,j] * (flow[i,j+1,1] if j<flow.shape[1]-1 else 0.0)
            
            flow[i,j,1] = (1-w) * flow[i,j,1] + w * ((b - Lx -Ux) / D)
            
    
    return flow

In [8]:
# Implements Horn-Schunck with both terms non-quadratic (for discontinuities and occlusions)

def lagged_diffusivity(gt, Ix, Iy, It, alpha, epsilon, err, num_it_sor):
    flow = np.ones((Ix.shape[0], Ix.shape[1], 2))
    
    for i in range(1000): 
    
        psi1 = 1 / np.sqrt((Ix[:,:,0]*flow[:,:,0] + Iy[:,:,0]*flow[:,:,1] + It[:,:,0])**2 + epsilon**2)

        flow_ip1 = np.vstack([flow[1:,:,:],flow[-1:,:,:]])
        flow_im1 = np.vstack([flow[:1,:,:],flow[:-1,:,:]])
        flow_jp1 = np.hstack([flow[:,1:,:],flow[:,-1:,:]])
        flow_jm1 = np.hstack([flow[:,:1,:],flow[:,:-1,:]])

        # norm of the gradient squared
        ngs = ((flow_ip1 - flow_im1) / 2)**2 + ((flow_jp1 - flow_jm1) / 2)**2

        ngs_ip1 = np.vstack([ngs[1:,:,:],ngs[-1:,:,:]])
        ngs_im1 = np.vstack([ngs[:1,:,:],ngs[:-1,:,:]])
        ngs_jp1 = np.hstack([ngs[:,1:,:],ngs[:,-1:,:]])
        ngs_jm1 = np.hstack([ngs[:,:1,:],ngs[:,:-1,:]])

        psi_im1 = 0.5 * ((1 / np.sqrt(ngs[:,:,0]+ngs[:,:,1]+epsilon**2)) + (1 / np.sqrt(ngs_im1[:,:,0]+ngs_im1[:,:,1]+epsilon**2)))
        psi_ip1 = 0.5 * ((1 / np.sqrt(ngs[:,:,0]+ngs[:,:,1]+epsilon**2)) + (1 / np.sqrt(ngs_ip1[:,:,0]+ngs_ip1[:,:,1]+epsilon**2)))
        psi_jm1 = 0.5 * ((1 / np.sqrt(ngs[:,:,0]+ngs[:,:,1]+epsilon**2)) + (1 / np.sqrt(ngs_jm1[:,:,0]+ngs_jm1[:,:,1]+epsilon**2)))
        psi_jp1 = 0.5 * ((1 / np.sqrt(ngs[:,:,0]+ngs[:,:,1]+epsilon**2)) + (1 / np.sqrt(ngs_jp1[:,:,0]+ngs_jp1[:,:,1]+epsilon**2)))

        
        for j in range(num_it_sor):
            flow = sor(Ix, Iy, It, psi1, psi_im1, psi_ip1, psi_jm1, psi_jp1, flow, alpha)
            
        if i % 100 == 0:
            print(aae(gt, flow.reshape((-1, 2))))
        
        if abs(err -  aae(gt, flow.reshape((-1, 2)))) < 0.0001:
            print('BYE, BYE', aae(gt, flow.reshape((-1, 2))))
            break
    
    return flow

In [9]:
flow = lagged_diffusivity(gt, Ix, Iy, It, 15.0, 0.001, 10.0, 10)
cv2.imwrite('result.png', cv2.cvtColor(flow2image(flow),cv2.COLOR_RGB2BGR))

53.962469713057445
55.09341505714063
55.09342382879568
55.093423832671206
55.09342383267099
55.09342383267099
55.09342383267099
55.09342383267099
55.09342383267099
55.09342383267099


True